# QSO Cl's

In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)

sys.path.insert(1, '/home/jaimerz/PhD/DEScls')

from xcell.mappers import MappereBOSSQSO
from xcell.mappers import MapperP15CMBK
from astropy.io import fits
from astropy.table import Table
import pyccl as ccl
import numpy as np
import pylab as plt
import pymaster as nmt
import healpy as hp

In [2]:
# Initialize binning scheme with 20 ells per bandpower
nside =  512
nside_mask = 512
#bands = nmt.NmtBin.from_nside_linear(nside, 20)
#ell_arr = bands.get_effective_ells()

In [3]:
ells = [0, 30, 60, 90, 120, 150, 180, 210, 240, 272, 309, 351, 398, 452, 513, 582, 661, 750, 852, 967, 1098, 1247, 1416, 1536] #1608, 1826, 2073, 2354, 2673, 3072]
bands = nmt.NmtBin.from_edges(ells[:-1], ells[1:])
ell_arr = bands.get_effective_ells()

In [4]:
bands = nmt.NmtBin.from_edges(ells[:-1], ells[1:])
ell_arr = bands.get_effective_ells()

In [5]:
NGC_1_config = {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-NGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-NGC-vDR16.fits'],
          'z_edges':[0, 1.5],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_NGC_1'
               }

NGC_2_config = {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-NGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-NGC-vDR16.fits'],
          'z_edges':[1.5, 3],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_NGC_2'}

SGC_1_config = {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-SGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-SGC-vDR16.fits'],
          'z_edges':[0, 1.5],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_SGC_1'}

SGC_2_config = {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-SGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-SGC-vDR16.fits'],
          'z_edges':[1.5, 3],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_SGC_1'}

ALL_1_config =  {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-NGC-vDR16.fits',
                                  '/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-SGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-NGC-vDR16.fits',
                             '/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-SGC-vDR16.fits'],
          'z_edges':[0, 1.5],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_ALL_1'}
ALL_2_config =  {'data_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-NGC-vDR16.fits',
                                  '/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_data-SGC-vDR16.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-NGC-vDR16.fits',
                             '/home/zcapjru/PhD/Data/eBOSS_QSO/eBOSS_QSO_clustering_random-SGC-vDR16.fits'],
          'z_edges':[1.5, 3],
          'nside':nside, 'nside_mask': nside_mask, 'mask_name': 'mask_QSO_ALL_2'}

In [6]:
maps_1 = MappereBOSSQSO(ALL_1_config)
maps_2 = MappereBOSSQSO(ALL_2_config)

In [7]:
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.049, h=0.67, sigma8=0.81, n_s=0.96)
b_ngc = 2.37*np.ones(50) 
b_sgc = 2.32 *np.ones(50) 

In [8]:
nz_1 = maps_1.get_nz()
nz_2 = maps_2.get_nz()

z_1 = nz_1[1]
dNdz_1 = nz_1[0]

z_2 = nz_2[1]
dNdz_2 = nz_2[0]

#quasar1 = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(z_1,dNdz_1), bias=(z_1,b_ngc))
#quasar2 = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(z_2,dNdz_2), bias=(z_2,b_ngc))
#CMB_shear = ccl.CMBLensingTracer(cosmo, 1100)

In [9]:
#ell = np.arange(2, 2000)
#theory_11 = ccl.angular_cl(cosmo, quasar1, quasar1, ell_arr) 
#theory_12 = ccl.angular_cl(cosmo, quasar1, quasar2, ell_arr) 
#theory_22 = ccl.angular_cl(cosmo, quasar2, quasar2, ell_arr) 

In [10]:
def compute_cl(f_a, f_b, wsp):
    # Compute the power spectrum (a la anafast) of the masked fields
    # Note that we only use n_iter=0 here to speed up the computation,
    # but the default value of 3 is recommended in general.
    cl_coupled = nmt.compute_coupled_cell(f_a, f_b)
    # Decouple power spectrum into bandpowers inverting the coupling matrix
    #cl_decoupled = wsp.decouple_cell(cl_coupled) #removed bias here

    return cl_coupled

In [11]:
f_1 =  maps_1.get_nmt_field()
f_2 =  maps_2.get_nmt_field()

/home/zcapjru/anaconda3/envs/PhD/lib/python3.8/site-packages/healpy/pixelfunc.py:475: RuntimeWarning: invalid value encountered in _ang2pix_ring
  return pixlib._ang2pix_ring(nside, theta, phi)


In [12]:
w_11 = nmt.NmtWorkspace()
w_11.compute_coupling_matrix(f_1, f_1, bands)

w_22 = nmt.NmtWorkspace()
w_22.compute_coupling_matrix(f_2, f_2, bands)

In [13]:
cl_c_11 = nmt.compute_coupled_cell(f_1, f_1)
cl_c_22 = nmt.compute_coupled_cell(f_2, f_2)

In [14]:
cl_11 = w_11.decouple_cell(cl_c_11)
cl_22 = w_22.decouple_cell(cl_c_22)

In [15]:
nl_c_11 = maps_1.get_nl_coupled() 
nl_c_22 = maps_2.get_nl_coupled() 

calculing nl from weights
calculing nl from weights


In [16]:
nl_11 = w_11.decouple_cell(nl_c_11)
nl_22 = w_22.decouple_cell(nl_c_22)

In [17]:
np.savez('512_eBOSSQSO_NGC+SGC_11', cl_coupled=cl_c_11, nl_decoupled=nl_11, nl_coupled=nl_c_11, cl_decoupled=cl_11)
np.savez('512_eBOSSQSO_NGC+SGC_22', cl_coupled=cl_c_22, nl_decoupled=nl_22, nl_coupled=nl_c_22, cl_decoupled=cl_22)

In [18]:
translate = {
    'NGC': 'N',
    'SGC': 'S',
    '0': 'l',
    '1': 'h'   
}

In [19]:
def load_david_cl(cup, a):
    cup = translate[cup]
    a = translate[a]
    return np.load('/home/zcapjru/PhD/Data/David_eBOSS/cls_{}{}_ns4096.npz'.format(cup, a))
def load_jaime_cl(cup, a):
    return np.loadtxt('/home/zcapjru/PhD/Cls/eBOSSQSO/4096/4096_{}_cl_{}{}.txt'.format(cup, a, a))
def load_jaime_nl(cup, a):
    return np.loadtxt('/home/zcapjru/PhD/Cls/eBOSSQSO/4096/4096_{}_nl_{}{}.txt'.format(cup, a, a))

In [20]:
NGC_nl_11[0]

NameError: name 'NGC_nl_11' is not defined

In [ ]:
plt.plot( NGC_nl_11[0], 'o-', label='cl')
plt.plot(load_david_cl('NGC', '1')['nl_coupled'][0], 'r--', label='david cl')
#plt.loglog()
plt.title('Quasars density angular NGC 22')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
cups = ['NGC', 'SGC']
for cup in cups:
    for i in range(2):
            david_ii = load_david_cl(cup,str(i))['cl'][0] - load_david_cl(cup,str(i))['nl'][0]
            jaime_ii = load_jaime_cl(cup,i)
            jaime_ii = jaime_ii - load_jaime_nl(cup,i)
            diff = abs(david_ii - jaime_ii)
            print('for {}_{}{} the max error is {}'.format(cup, i , i, max(diff)))

In [ ]:
load_david_cl('NGC', '1')['nl_coupled'][0]
#(ells > 2000) & (ells < 8000)

In [ ]:
plt.plot( load_jaime_nl('SGC', 0) , 'bs', label='Jaime nl')
plt.plot( load_jaime_cl('SGC', 0) , 'yo', label='Jaime Cl')
#plt.semilogx()
plt.title('Comp')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='upper right', ncol=2, labelspacing=0.1)
plt.ylim([9e-6, 1.05e-5])
plt.show()

In [ ]:
plt.plot( load_jaime_nl('SGC', 1) , 'bs', label='Jaime nl')
plt.plot( load_jaime_cl('SGC', 1) , 'yo', label='Jaime Cl')
#plt.semilogx()
plt.title('Comp')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='upper right', ncol=2, labelspacing=0.1)
plt.ylim([9e-6, 1.05e-5])
plt.show()

In [ ]:
plt.plot(ell_arr, load_david_cl('NGC', '1')['cl'][0]-  load_david_cl('NGC', '1')['nl'][0], 'bs', label='David Cl')
plt.plot(ell_arr, load_jaime_cl('NGC', 1) - load_jaime_nl('NGC',1) , 'yo', label='Jaime Cl')
plt.loglog()
plt.title('Comp')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr,   load_david_cl('NGC', '1')['nl'][0], 'bs', label='David nl')
plt.plot(ell_arr,  load_jaime_nl('NGC',1) , 'yo', label='Jaime nl')
plt.loglog()
plt.title('Comp')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
load_david_cl('NGC', '1')['ls']

In [ ]:
ell_arr